In [2]:
import json
import pandas as pd
from IPython.display import display, Markdown

def read_experiment_data(file_path):
    """读取实验数据文件"""
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    return data


def extract_experiment_info(experiments):
    """提取实验信息"""
    results = []

    for exp in experiments:
        config = exp.get("config", {})
        result = exp.get("result", {})

        # 提取所需的参数
        eval_dataset = config.get("eval_dataset", "N/A")
        edge_cloud_bandwidth = config.get("edge_cloud_bandwidth", 0)
        edge_end_bandwidth = config.get("edge_end_bandwidth", 0)
        eval_mode = result.get("eval_mode", "")
        accuracy = result.get("accuracy", "N/A")  # 提取 accuracy
        target_forward_times = result.get("target_forward_times", 0)
        generated_tokens = result.get("generated_tokens", 0)
        wall_time = result.get("wall_time", 0)
        communication_time = result.get("communication_time", 0)
        queuing_time = result.get("queuing_time", 0)
        edge_cloud_data_bytes = result.get("edge_cloud_data_bytes", 0)
        throughput = result.get("throughput", 0)
        comm_energy = result.get("comm_energy", 0)

        results.append(
            {
                "数据集": eval_dataset,
                "Edge-Cloud 带宽 (Mbps)": edge_cloud_bandwidth,
                "Edge-End 带宽 (Mbps)": edge_end_bandwidth,
                "方法": eval_mode,
                "准确率": accuracy,  # 添加到结果中
                "生成 Token 数": generated_tokens,
                "目标模型 Forward 次数": target_forward_times,
                "挂钟时间 (s)": round(wall_time, 2),
                "通信时间 (s)": round(communication_time, 2),
                "排队时间 (s)": round(queuing_time, 2),
                "Edge-Cloud 数据传输量 (bytes)": edge_cloud_data_bytes,
                "Token 输出速率 (tokens/s)": round(throughput, 2),
                "通信能量": comm_energy,
            }
        )

    return results

def process_data(experiment_data):
    """处理实验数据，计算相对算力消耗并转换为DataFrame"""
    df = pd.DataFrame(experiment_data)

    # 找到 large 方法的基准值
    # 假设每个数据集都有一个 large 实验作为基准
    large_df = df[df['方法'] == 'large'][['数据集', '目标模型 Forward 次数']].drop_duplicates(subset=['数据集'])
    large_df = large_df.rename(columns={'目标模型 Forward 次数': 'baseline_forwards'})

    # 合并基准值
    df = pd.merge(df, large_df, on='数据集', how='left')

    # 计算相对 large 算力消耗 (%)
    df['相对 large 算力消耗 (%)'] = df.apply(
        lambda row: round((row['目标模型 Forward 次数'] / row['baseline_forwards'] * 100), 2)
        if pd.notnull(row['baseline_forwards']) and row['baseline_forwards'] > 0
        else None,
        axis=1
    )
    
    # 核心修改：计算 Generation Offloading Rate (%)
    # 公式：1 - target_forward_times / generated_tokens
    df['Generation Offloading Rate (%)'] = df.apply(
        lambda row: round((1 - row['目标模型 Forward 次数'] / row['生成 Token 数']) * 100, 2)
        if row['生成 Token 数'] > 0 else 0,
        axis=1
    )


    
    # 清理临时列
    if 'baseline_forwards' in df.columns:
        del df['baseline_forwards']
    
    return df

def generate_markdown_table(df):
    """生成 markdown 表格"""
    # 按方法分组，然后按Edge-Cloud带宽降序排序
    df_sorted = df.sort_values(
        ["方法", "Edge-Cloud 带宽 (Mbps)"], ascending=[True, False]
    )

    # 生成 markdown 表格
    markdown_table = df_sorted.to_markdown(index=False, tablefmt="github")
    return markdown_table


def get_pandas_table(df):
    """生成 pandas DataFrame 表格"""
    # 按方法分组，然后按Edge-Cloud带宽降序排序
    df_sorted = df.sort_values(
        ["方法", "Edge-Cloud 带宽 (Mbps)"], ascending=[True, False]
    )
    return df_sorted


def generate_grouped_tables(df):
    """按方法分组生成多个表格"""
    # 按方法分组
    grouped = df.groupby("方法")

    all_tables = []

    for method, group_df in grouped:
        # 按Edge-Cloud带宽降序排序
        group_df_sorted = group_df.sort_values(
            "Edge-Cloud 带宽 (Mbps)", ascending=False
        )

        # 生成表格
        table = group_df_sorted.to_markdown(index=False, tablefmt="github")

        all_tables.append(f"## 方法: {method}\n\n{table}\n")

    return "\n".join(all_tables)


def generate_dataset_grouped_tables(df):
    """按数据集分组生成多个表格"""
    # 按数据集分组
    grouped = df.groupby("数据集")

    all_tables = []

    for dataset, group_df in grouped:
        # 排序：按方法排序，带宽降序
        group_df_sorted = group_df.sort_values(
            ["方法", "Edge-Cloud 带宽 (Mbps)"], ascending=[True, False]
        )

        # 生成表格
        table = group_df_sorted.to_markdown(index=False, tablefmt="github")

        all_tables.append(f"## 数据集: {dataset}\n\n{table}\n")

    return "\n".join(all_tables)


def jupyter_display(df):
    """
    一个专门为 Jupyter Notebook 设计的主函数，
    用于读取、处理数据并以美观的表格形式显示结果。
    """
    # --- 1. 显示统一排序的表格 ---
    # 使用 display(Markdown(...)) 来渲染标题
    display(Markdown("# 实验结果对比表（按方法和带宽排序）"))

    # 对 DataFrame 进行排序
    df_sorted = df.sort_values(
        ["方法", "Edge-Cloud 带宽 (Mbps)"], ascending=[True, False]
    )

    # 直接显示 DataFrame 对象，Jupyter 会将其渲染为 HTML 表格
    display(df_sorted)
    
    # --- 2. 显示按方法分组的表格 ---
    display(Markdown("# 实验结果对比表（按方法分组）"))

    # 按方法进行分组
    grouped = df.groupby("方法")
    # 遍历每个分组并分别显示
    for method, group_df in grouped:
        # 显示分组标题
        display(Markdown(f"## 方法: {method}"))

        # 对当前分组的 DataFrame 进行排序
        group_df_sorted = group_df.sort_values(
            "Edge-Cloud 带宽 (Mbps)", ascending=False
        )

        # 显示排序后的分组 DataFrame
        display(group_df_sorted)

    # --- 3. 显示按数据集分组的表格 ---
    display(Markdown("# 实验结果对比表（按数据集分组）"))

    # 按数据集进行分组
    grouped_ds = df.groupby("数据集")
    for dataset, group_df in grouped_ds:
        # 显示分组标题
        display(Markdown(f"## 数据集: {dataset}"))

        # 排序：按方法排序，带宽降序
        group_df_sorted = group_df.sort_values(
            ["方法", "Edge-Cloud 带宽 (Mbps)"], ascending=[True, False]
        )

        # 显示排序后的分组 DataFrame
        display(group_df_sorted)


def main(specified_file=None):
    # 读取实验数据
    import glob
    import os
    
    file_path = None
    
    # 1. 优先使用用户指定的文件
    if specified_file:
        if os.path.exists(specified_file):
            file_path = specified_file
        else:
            print(f"指定的文件不存在: {specified_file}")
            return

    # 2. 如果没有指定文件，则寻找最新的 experiment_summary_*.json
    if not file_path:
        json_files = glob.glob("experiment_summary_*.json")
        if json_files:
            file_path = max(json_files, key=os.path.getmtime)
        else:
            # 3. 最后的兜底
            target_file = "experiment_summary_20260127_053240.json"
            if os.path.exists(target_file):
                file_path = target_file
            else:
                print("找不到任何实验总结 JSON 文件")
                return
            
    print(f"处理文件: {file_path}")

    experiments = read_experiment_data(file_path)
    experiment_data = extract_experiment_info(experiments)

    # process data to dataframe and add new metrics
    df = process_data(experiment_data)

    unified_table = generate_markdown_table(df)
    method_grouped_tables = generate_grouped_tables(df)
    dataset_grouped_tables = generate_dataset_grouped_tables(df)

    jupyter_display(df)

    # 保存到文件
    with open("experiment_results_table.md", "w", encoding="utf-8") as f:
        f.write("# 实验结果对比表\n\n")
        f.write("## 1. 统一对比表（按方法和带宽排序）\n\n")
        f.write(unified_table)
        f.write("\n\n" + "=" * 80 + "\n\n")
        f.write("## 2. 按方法分组表格\n\n")
        f.write(method_grouped_tables)
        f.write("\n\n" + "=" * 80 + "\n\n")
        f.write("## 3. 按数据集分组表格\n\n")
        f.write(dataset_grouped_tables)

    print("\n表格已保存到 experiment_results_table.md 文件中")


if __name__ == "__main__":
    # 如果想指定文件，请修改下一行，例如: main("experiment_summary_20260127_053240.json")
    main("experiment_summary_20260126_153921.json")

处理文件: experiment_summary_20260126_153921.json


# 实验结果对比表（按方法和带宽排序）

,数据集,Edge-Cloud 带宽 (Mbps),Edge-End 带宽 (Mbps),方法,准确率,生成 Token 数,目标模型 Forward 次数,挂钟时间 (s),通信时间 (s),排队时间 (s),Edge-Cloud 数据传输量 (bytes),Token 输出速率 (tokens/s),通信能量,相对 large 算力消耗 (%),Generation Offloading Rate (%)
20,eval/eval_humaneval.py,34.6,563,adaptive_tridecoding,0.325,10428,2626,583.16,0.30,131.30,664292,17.88,0.0,25.64,74.82
21,eval/eval_mt_bench_noeval.py,34.6,563,adaptive_tridecoding,0,20928,5335,1149.23,0.45,266.75,1831926,18.21,0.0,26.05,74.51
22,eval/eval_cnndm.py,34.6,563,adaptive_tridecoding,"{'rouge1': 0.2818606062036901, 'rouge2': 0.124...",10499,2068,559.05,0.87,103.40,886894,18.78,0.0,20.20,80.30
23,eval/eval_xsum.py,34.6,563,adaptive_tridecoding,"{'rouge1': 0.1448398875610998, 'rouge2': 0.028...",10600,2131,537.17,0.75,106.55,833254,19.73,0.0,20.81,79.90
24,eval/eval_gsm8k.py,34.6,563,adaptive_tridecoding,0.0125,10541,2361,524.00,0.29,118.05,701316,20.12,0.0,23.06,77.60
10,eval/eval_humaneval.py,34.6,563,dist_spec,0.325,10240,2500,402.33,2.64,125.00,9862202,25.45,0.0,24.41,75.59
11,eval/eval_mt_bench_noeval.py,34.6,563,dist_spec,0,20480,5329,855.33,5.98,266.45,25857254,23.94,0.0,26.02,73.98
12,eval/eval_cnndm.py,34.6,563,dist_spec,"{'rouge1': 0.28424497032430085, 'rouge2': 0.12...",10240,2370,407.21,5.92,118.50,20671598,25.15,0.0,23.14,76.86
13,eval/eval_xsum.py,34.6,563,dist_spec,"{'rouge1': 0.14709188110710264, 'rouge2': 0.02...",10240,2403,406.89,5.35,120.15,18618190,25.17,0.0,23.47,76.53
14,eval/eval_gsm8k.py,34.6,563,dist_spec,0.0125,10240,2563,405.11,2.58,128.15,9584112,25.28,0.0,25.03,74.97


# 实验结果对比表（按方法分组）

## 方法: adaptive_tridecoding

...

## 数据集: eval/eval_mt_bench_noeval.py

,数据集,Edge-Cloud 带宽 (Mbps),Edge-End 带宽 (Mbps),方法,准确率,生成 Token 数,目标模型 Forward 次数,挂钟时间 (s),通信时间 (s),排队时间 (s),Edge-Cloud 数据传输量 (bytes),Token 输出速率 (tokens/s),通信能量,相对 large 算力消耗 (%),Generation Offloading Rate (%)
21,eval/eval_mt_bench_noeval.py,34.6,563,adaptive_tridecoding,0,20928,5335,1149.23,0.45,266.75,1831926,18.21,0.0,26.05,74.51
11,eval/eval_mt_bench_noeval.py,34.6,563,dist_spec,0,20480,5329,855.33,5.98,266.45,25857254,23.94,0.0,26.02,73.98
6,eval/eval_mt_bench_noeval.py,34.6,563,dist_split_spec,0,20480,5329,847.27,0.33,266.45,1421306,24.17,0.0,26.02,73.98
1,eval/eval_mt_bench_noeval.py,34.6,563,large,0,20480,20480,1950.96,0.00,1024.00,0,10.50,0.0,100.00,0.00
16,eval/eval_mt_bench_noeval.py,34.6,563,uncertainty_decoding,0,20480,561,1198.60,6.28,518.55,27151766,17.09,0.0,2.74,97.26


## 数据集: eval/eval_xsum.py

,数据集,Edge-Cloud 带宽 (Mbps),Edge-End 带宽 (Mbps),方法,准确率,生成 Token 数,目标模型 Forward 次数,挂钟时间 (s),通信时间 (s),排队时间 (s),Edge-Cloud 数据传输量 (bytes),Token 输出速率 (tokens/s),通信能量,相对 large 算力消耗 (%),Generation Offloading Rate (%)
23,eval/eval_xsum.py,34.6,563,adaptive_tridecoding,"{'rouge1': 0.1448398875610998, 'rouge2': 0.028...",10600,2131,537.17,0.75,106.55,833254,19.73,0.0,20.81,79.90
13,eval/eval_xsum.py,34.6,563,dist_spec,"{'rouge1': 0.14709188110710264, 'rouge2': 0.02...",10240,2403,406.89,5.35,120.15,18618190,25.17,0.0,23.47,76.53
8,eval/eval_xsum.py,34.6,563,dist_split_spec,"{'rouge1': 0.14709188110710264, 'rouge2': 0.02...",10240,2403,400.57,0.11,120.15,411298,25.56,0.0,23.47,76.53
3,eval/eval_xsum.py,34.6,563,large,"{'rouge1': 0.14265407624458998, 'rouge2': 0.02...",10240,10240,997.96,0.00,512.00,0,10.26,0.0,100.00,0.00
18,eval/eval_xsum.py,34.6,563,uncertainty_decoding,"{'rouge1': 0.1497173225813533, 'rouge2': 0.029...",10240,269,627.03,8.05,260.00,27923540,16.33,0.0,2.63,97.37



表格已保存到 experiment_results_table.md 文件中
